In [1]:
!pip install polars xlsx2csv xlsxwriter seaborn pyarrow numpy datapackage duckdb altair

  Obtaining dependency information for altair from https://files.pythonhosted.org/packages/c5/e4/7fcceef127badbb0d644d730d992410e4f3799b295c9964a172f92a469c7/altair-5.2.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.9/996.9 kB 7.8 MB/s eta 0:00:00a 0:00:01


In [10]:
import polars as pl
# DENNA laddade jag ner från https://www.kaggle.com/datasets/ruchi798/currency-exchange-rates och gjorde om till excel fil för demo
pl.read_excel("./data/currencies.xlsx")

Column1,Country/Currency,currency,value,date
i64,str,str,f64,str
0,"""Australia Doll…","""AUD""",1.581627,"""17/12/2021"""
1,"""Great Britain …","""GBP""",0.851619,"""17/12/2021"""
2,"""Euro""","""EUR""",1.0,"""17/12/2021"""
3,"""Japan Yen""","""JPY""",128.301759,"""17/12/2021"""
4,"""Switzerland Fr…","""CHF""",1.041015,"""17/12/2021"""
5,"""USA Dollar""","""USD""",1.132548,"""17/12/2021"""
6,"""Afghanistan Af…","""AFN""",115.81055,"""17/12/2021"""
7,"""Albania Lek""","""ALL""",120.855126,"""17/12/2021"""
8,"""Algeria Dinar""","""DZD""",157.243849,"""17/12/2021"""


In [16]:
pl.read_csv("./data/exchange_rates.csv").with_columns(pl.col('date').str.to_datetime("%d/%m/%Y")).write_parquet("./data/currencies.parquet")

In [27]:
import datapackage


data_url = 'https://datahub.io/core/currency-codes/datapackage.json'

# to load Data Package into storage
package = datapackage.Package(data_url)

# to load only tabular data
resources = package.resources
for resource in resources:
    if resource.tabular:
        data = pl.read_csv(resource.descriptor['path'], null_values=['-'])
        data.write_parquet("output/currency_codes_iso4217.parquet")

In [31]:
data_url = 'https://datahub.io/core/country-codes/datapackage.json'

# to load Data Package into storage
package = datapackage.Package(data_url)

# to load only tabular data
resources = package.resources
for resource in resources:
    if resource.tabular:
        data = pl.read_csv(resource.descriptor['path'],infer_schema_length=10000)
        print(data.columns)
        data.write_parquet("output/country_info_iso3666.parquet")

['FIFA', 'Dial', 'ISO3166-1-Alpha-3', 'MARC', 'is_independent', 'ISO3166-1-numeric', 'GAUL', 'FIPS', 'WMO', 'ISO3166-1-Alpha-2', 'ITU', 'IOC', 'DS', 'UNTERM Spanish Formal', 'Global Code', 'Intermediate Region Code', 'official_name_fr', 'UNTERM French Short', 'ISO4217-currency_name', 'Developed / Developing Countries', 'UNTERM Russian Formal', 'UNTERM English Short', 'ISO4217-currency_alphabetic_code', 'Small Island Developing States (SIDS)', 'UNTERM Spanish Short', 'ISO4217-currency_numeric_code', 'UNTERM Chinese Formal', 'UNTERM French Formal', 'UNTERM Russian Short', 'M49', 'Sub-region Code', 'Region Code', 'official_name_ar', 'ISO4217-currency_minor_unit', 'UNTERM Arabic Formal', 'UNTERM Chinese Short', 'Land Locked Developing Countries (LLDC)', 'Intermediate Region Name', 'official_name_es', 'UNTERM English Formal', 'official_name_cn', 'official_name_en', 'ISO4217-currency_country_name', 'Least Developed Countries (LDC)', 'Region Name', 'UNTERM Arabic Short', 'Sub-region Name', 'o